In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
from scipy import stats
import matplotlib.pyplot as plt
from datetime import datetime
from tqdm import tqdm
from sklearn.preprocessing import StandardScaler
import networkx as nx
import warnings
warnings.filterwarnings('ignore')

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Seting the directory 
%cd /content/drive/MyDrive/TP_M1_SID/Second_semestre/Compet_kaggle  

/content/drive/MyDrive/TP_M1_SID/Second_semestre/Compet_kaggle


# Studying the relationship between the different authors.

I got the intuition that the more an author  'is known' the more his comments are likely to have a high score. 

In [ ]:
df = pd.read_csv("comments_students.csv")[['id','ups', 'link_id','name', 'author', 'parent_id']]
df.head(15) 

,id,ups,link_id,name,author,parent_id
0,cqug90j,3.0,t3_34f9rh,t1_cqug90j,jesse9o3,t1_cqug2sr
1,cqug90k,3.0,t3_34fvry,t1_cqug90k,beltfedshooter,t3_34fvry
2,cqug90z,5.0,t3_34ffo5,t1_cqug90z,InterimFatGuy,t1_cqu80zb
3,cqug91c,1.0,t3_34aqsn,t1_cqug91c,JuanTutrego,t1_cqtdj4m
4,cqug91e,101.0,t3_34f9rh,t1_cqug91e,dcblackbelt,t1_cquc4rc
5,cqug920,1.0,t3_34fyop,t1_cqug920,TheDoorsShirt,t3_34fyop
6,cqug921,-5.0,t3_34fvry,t1_cqug921,youthfulvictim,t1_cqug6hk
7,cqug923,1.0,t3_34glje,t1_cqug923,sonovadoyle,t1_cqug6r6
8,cqug929,1.0,t3_34gmo3,t1_cqug929,boludo54,t3_34gmo3
9,cqug92o,1.0,t3_34glt0,t1_cqug92o,mister_sleepy,t3_34glt0


In [ ]:
df.shape 

(4234970, 6)

In this code cell, I have selected all the parent_id and i have to look for the names of the comments which correspond, that allows us to obtain the author of the "parent comments". 

In [ ]:
# creating a list of all parent_id 
is_parent_id=df['parent_id'].values.tolist()
# selecting a subdataframe, that contains only the name comments which is in list of parent_id 
df_parent=df[df['name'].isin(is_parent_id)][['name','author']]
df_parent.rename(columns={'name':'parent_id','author':'parent_author'},inplace=True)
df_parent.head(10) 

,parent_id,parent_author
4,t1_cqug91e,dcblackbelt
6,t1_cqug921,youthfulvictim
7,t1_cqug923,sonovadoyle
15,t1_cqug93u,bunnylumps
18,t1_cqug94d,SpareLiver
22,t1_cqug94y,robondes
24,t1_cqug95f,JustMe80
25,t1_cqug966,wackawacka2
37,t1_cqug99q,paigenotcasey
40,t1_cqug9ad,battlefieldsucks


In [ ]:
df_parent.shape

(1304791, 2)

In [ ]:
# creating a new column "parent_author" by merging the dataframe df_parent and the initial dataframe 
df=df.merge(df_parent,how='outer')
df

,id,ups,link_id,name,author,parent_id,parent_author
0,cqug90j,3.0,t3_34f9rh,t1_cqug90j,jesse9o3,t1_cqug2sr,NaN
1,cqug90k,3.0,t3_34fvry,t1_cqug90k,beltfedshooter,t3_34fvry,NaN
2,cqug9uh,-1.0,t3_34fvry,t1_cqug9uh,nfleite,t3_34fvry,NaN
3,cquga7w,0.0,t3_34fvry,t1_cquga7w,[deleted],t3_34fvry,NaN
4,cqugbfk,-1.0,t3_34fvry,t1_cqugbfk,uniballer2007,t3_34fvry,NaN
...,...,...,...,...,...,...,...
4234965,crrbedv,NaN,t3_380m0r,t1_crrbedv,Ahmed-kabbary2015,t1_crrb4tv,Phe0niKs
4234966,crrbego,NaN,t3_380mst,t1_crrbego,vokabulary,t3_380mst,NaN
4234967,crrbek0,NaN,t3_380i9k,t1_crrbek0,fehrihl,t1_crra53y,HatchetToGather
4234968,crrbelf,NaN,t3_37y8kp,t1_crrbelf,YoungAdult_,t1_crr99vg,MephistoSchreck


In [ ]:
df_non_deleted = df[(df['author'] !='[deleted]') & (df['parent_author'] !='[deleted]')] 

In [ ]:
df_non_deleted 

,id,ups,link_id,name,author,parent_id,parent_author
0,cqug90j,3.0,t3_34f9rh,t1_cqug90j,jesse9o3,t1_cqug2sr,NaN
1,cqug90k,3.0,t3_34fvry,t1_cqug90k,beltfedshooter,t3_34fvry,NaN
2,cqug9uh,-1.0,t3_34fvry,t1_cqug9uh,nfleite,t3_34fvry,NaN
4,cqugbfk,-1.0,t3_34fvry,t1_cqugbfk,uniballer2007,t3_34fvry,NaN
5,cqugc6d,1.0,t3_34fvry,t1_cqugc6d,Megaman915,t3_34fvry,NaN
...,...,...,...,...,...,...,...
4234965,crrbedv,NaN,t3_380m0r,t1_crrbedv,Ahmed-kabbary2015,t1_crrb4tv,Phe0niKs
4234966,crrbego,NaN,t3_380mst,t1_crrbego,vokabulary,t3_380mst,NaN
4234967,crrbek0,NaN,t3_380i9k,t1_crrbek0,fehrihl,t1_crra53y,HatchetToGather
4234968,crrbelf,NaN,t3_37y8kp,t1_crrbelf,YoungAdult_,t1_crr99vg,MephistoSchreck


In [ ]:
df_non_deleted['parent_author'].mode()[0] 

'Late_Night_Grumbler'

In [ ]:
df_non_deleted['author'].mode()[0]

'AutoModerator'

I notice that there is a large number of comments whose author is a '[deleted]' and somme of 'parent_author' are NAN. So we chose to replace '[deleted]' values by the most frequent author, and nan by the most frequent parent_author.  

In [ ]:
df['parent_author'].fillna(df_non_deleted['parent_author'].mode()[0],inplace=True) 
df['author'].replace('[deleted]', df_non_deleted['author'].mode()[0], inplace = True)
df

,id,ups,link_id,name,author,parent_id,parent_author
0,cqug90j,3.0,t3_34f9rh,t1_cqug90j,jesse9o3,t1_cqug2sr,Late_Night_Grumbler
1,cqug90k,3.0,t3_34fvry,t1_cqug90k,beltfedshooter,t3_34fvry,Late_Night_Grumbler
2,cqug9uh,-1.0,t3_34fvry,t1_cqug9uh,nfleite,t3_34fvry,Late_Night_Grumbler
3,cquga7w,0.0,t3_34fvry,t1_cquga7w,AutoModerator,t3_34fvry,Late_Night_Grumbler
4,cqugbfk,-1.0,t3_34fvry,t1_cqugbfk,uniballer2007,t3_34fvry,Late_Night_Grumbler
...,...,...,...,...,...,...,...
4234965,crrbedv,NaN,t3_380m0r,t1_crrbedv,Ahmed-kabbary2015,t1_crrb4tv,Phe0niKs
4234966,crrbego,NaN,t3_380mst,t1_crrbego,vokabulary,t3_380mst,Late_Night_Grumbler
4234967,crrbek0,NaN,t3_380i9k,t1_crrbek0,fehrihl,t1_crra53y,HatchetToGather
4234968,crrbelf,NaN,t3_37y8kp,t1_crrbelf,YoungAdult_,t1_crr99vg,MephistoSchreck


# Creating a graph
I created a directed graph where the nodes are the authors and there is an edge between two authors if one is the parent_author of the other. 

In [ ]:
G = nx.DiGraph()

In [ ]:
G.add_nodes_from(df['author'].values)

In [ ]:
%time 
G.add_edges_from(df[["parent_author", "author"]].values)  
  

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 5.25 µs


In [ ]:
print(G.number_of_nodes())
print(G.number_of_edges()) 

570735
2501670


# Succesors and predecessors


In [ ]:
df["successors_auth"] = [len(list(G.successors(i))) for i in df["author"]]

In [ ]:
df["predecessors_auth"] = [len(list(G.predecessors(i))) for i in df["author"]]  

# Centralities

In [ ]:
def val_ctr(id):
  return centrality[id] 

### 1. Degree
I assume that the more an author has interacted with other authors, the more likely he is to be known, thus getting a high score for his comments.

In [ ]:
centrality = nx.degree_centrality(G)

In [ ]:
df['Ctr_dg_autor'] = df['author'].apply(func = val_ctr) 

### 2. Eigen vector

I suppose that the more an author interacts with an author who has good scores, the more likely he is to be known, thus obtaining a high score for his comments.

In [ ]:
%%time
centrality = nx.eigenvector_centrality(G) 

CPU times: user 40.5 s, sys: 284 ms, total: 40.8 s
Wall time: 40.8 s


In [ ]:
df['Ctr_Eigen_auth'] = df['author'].apply(func = val_ctr)
del centrality 

## Prestige

### 1. pageRank
I calculate the importance of an author according to the importance of the authors who point to him (who respond to his comments).

In [ ]:
def val_ctr(id):
  return pgR[id]

In [ ]:
pgR = nx.pagerank_scipy(G)

In [ ]:
df['page_rk_auth'] = df['author'].apply(func = val_ctr)
del pgR 

# Calculating the frequency of occurrence of an author
I assign a weight to each author according to his frequency of appearance, assuming that the more comments an author writes the more likely he is to be known.

In [ ]:
import collections as clt

lst_authors=df['author'].values.tolist()
occurrence_author = clt.Counter()
occ_author=clt.Counter()
for elt in lst_authors:
    occ_author[elt]+=1



In [ ]:
def freq_occurrence(author):
  n = len(df)
  return occ_author[author]/n 

In [ ]:
df['freq_occ_author']=df['author'].apply(freq_occurrence)

In [ ]:
df

,id,ups,link_id,name,author,parent_id,parent_author,successors_auth,predecessors_auth,Ctr_dg_autor,Ctr_Eigen_auth,page_rk_auth,freq_occ_author
0,cqug90j,3.0,t3_34f9rh,t1_cqug90j,jesse9o3,t1_cqug2sr,Late_Night_Grumbler,42,40,0.000144,0.001702,0.000015,1.275098e-05
1,cqug90k,3.0,t3_34fvry,t1_cqug90k,beltfedshooter,t3_34fvry,Late_Night_Grumbler,0,4,0.000007,0.000621,0.000001,1.180646e-06
2,cqug9uh,-1.0,t3_34fvry,t1_cqug9uh,nfleite,t3_34fvry,Late_Night_Grumbler,4,7,0.000019,0.000705,0.000003,4.958713e-06
3,cquga7w,0.0,t3_34fvry,t1_cquga7w,AutoModerator,t3_34fvry,Late_Night_Grumbler,235,62645,0.110174,0.815823,0.020974,8.238949e-02
4,cqugbfk,-1.0,t3_34fvry,t1_cqugbfk,uniballer2007,t3_34fvry,Late_Night_Grumbler,2,2,0.000007,0.000580,0.000002,4.722584e-07
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4234965,crrbedv,NaN,t3_380m0r,t1_crrbedv,Ahmed-kabbary2015,t1_crrb4tv,Phe0niKs,8,13,0.000037,0.000990,0.000006,1.062581e-05
4234966,crrbego,NaN,t3_380mst,t1_crrbego,vokabulary,t3_380mst,Late_Night_Grumbler,2,8,0.000018,0.000634,0.000005,2.833550e-06
4234967,crrbek0,NaN,t3_380i9k,t1_crrbek0,fehrihl,t1_crra53y,HatchetToGather,3,9,0.000021,0.000766,0.000004,3.305809e-06
4234968,crrbelf,NaN,t3_37y8kp,t1_crrbelf,YoungAdult_,t1_crr99vg,MephistoSchreck,97,84,0.000317,0.002838,0.000012,2.266840e-05


## Scaling data 
I scale data to avoid the problem of discrimination between different individuals when applying the machine learning model

In [ ]:
scaler = StandardScaler()
df.iloc[:,7:]=scaler.fit_transform(df.iloc[:,7:])
df 

,id,ups,link_id,name,author,parent_id,parent_author,successors_auth,predecessors_auth,Ctr_dg_autor,Ctr_Eigen_auth,page_rk_auth,freq_occ_author
0,cqug90j,3.0,t3_34f9rh,t1_cqug90j,jesse9o3,t1_cqug2sr,Late_Night_Grumbler,-0.046253,-0.300397,-0.247372,-0.302522,-0.299207,-0.300596
1,cqug90k,3.0,t3_34fvry,t1_cqug90k,beltfedshooter,t3_34fvry,Late_Night_Grumbler,-0.048731,-0.302489,-0.250612,-0.307353,-0.301532,-0.301107
2,cqug9uh,-1.0,t3_34fvry,t1_cqug9uh,nfleite,t3_34fvry,Late_Night_Grumbler,-0.048495,-0.302314,-0.250322,-0.306977,-0.301189,-0.300941
3,cquga7w,0.0,t3_34fvry,t1_cquga7w,AutoModerator,t3_34fvry,Late_Night_Grumbler,-0.034864,3.337207,2.361777,3.336656,3.337246,3.337239
4,cqugbfk,-1.0,t3_34fvry,t1_cqugbfk,uniballer2007,t3_34fvry,Late_Night_Grumbler,-0.048613,-0.302605,-0.250612,-0.307535,-0.301468,-0.301139
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4234965,crrbedv,NaN,t3_380m0r,t1_crrbedv,Ahmed-kabbary2015,t1_crrb4tv,Phe0niKs,-0.048259,-0.301966,-0.249906,-0.305704,-0.300768,-0.300690
4234966,crrbego,NaN,t3_380mst,t1_crrbego,vokabulary,t3_380mst,Late_Night_Grumbler,-0.048613,-0.302256,-0.250363,-0.307296,-0.300846,-0.301034
4234967,crrbek0,NaN,t3_380i9k,t1_crrbek0,fehrihl,t1_crra53y,HatchetToGather,-0.048554,-0.302198,-0.250280,-0.306705,-0.300990,-0.301014
4234968,crrbelf,NaN,t3_37y8kp,t1_crrbelf,YoungAdult_,t1_crr99vg,MephistoSchreck,-0.043007,-0.297840,-0.243258,-0.297440,-0.299695,-0.300159


## Saving scaled data

In [ ]:
df.to_csv('df_author_scaled.csv', index= False)